In [19]:
import pandas as pd

from orion.data import load_signal
from orion import Orion
from orion.data import load_anomalies

from mlprimitives.custom.timeseries_preprocessing import time_segments_aggregate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from mlprimitives.custom.timeseries_preprocessing import rolling_window_sequences
from orion.primitives.timeseries_preprocessing import slice_array_by_dims
from mlprimitives import load_primitive

from orion.evaluation.contextual import contextual_f1_score
import numpy as np
import pickle

import os, sys
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

from src.configuration.constants import *

In [7]:
with open(os.path.join(PROCESSED_DATA_DIRECTORY, 'output.pickle'), 'rb') as f:
    output = pickle.load(f)
    
output.keys()

dict_keys(['X_test', 'y_test', 'index_test', 'y_hat', 'critic'])

In [20]:
with open(os.path.join(INTERIM_DATA_DIRECTORY, 'MSL.pickle'), 'rb') as f:
    test = pickle.load(f)

In [35]:
ground_truth = test['anomaly'][test['anomaly'].signal=='M-1'][['start', 'end']]

In [9]:
y = output['y_test']
index = output['index_test']

critic = output['critic']
y_hat = output['y_hat']

In [10]:
# computes an array of anomaly scores based on a combination of reconstruction error and critic output
params = {"rec_error_type": "dtw"}

primitive = load_primitive("orion.primitives.tadgan.score_anomalies", 
                           arguments=params)
errors, true_index, true, predictions = primitive.produce(y=y, y_hat=y_hat, critic=critic, index=index)

errors.shape, true_index.shape

Using TensorFlow backend.


((2276,), (2177,))

In [11]:
print("average error value: {:.2f}".format(errors.mean()))

average error value: 2.86


In [46]:
# extracts anomalies from sequences of errors following the approach
params = {
    "window_size_portion": 0.05, 
    "window_step_size_portion": 0.1,
    "fixed_threshold": True
}

primitive = load_primitive("orion.primitives.timeseries_anomalies.find_anomalies", 
                           arguments=params)
e = primitive.produce(errors=errors, index=true_index)

e.shape

(3, 3)

In [47]:
anomalies = [(int(i[0]), int(i[1])) for i in e]

In [48]:
start, end = index[0], index[-1]
contextual_f1_score(ground_truth, anomalies, start=start, end=end, weighted=True)

0.07721423164269492

In [49]:
ground_truth

,start,end
0,1110,2250


In [50]:
anomalies

[(132, 195), (265, 329), (1272, 1322)]